In [1]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

  # Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [2]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [3]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [4]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [5]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,9,4200.0,37800,190000.0,NaN,No,District
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,NaN,NaN,No,District
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,9600.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
383,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District
384,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District
385,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District
386,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District


In [6]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [7]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [8]:
district_df["NAME OF THE DISTRICT"].nunique()

61

In [9]:
district_df["REQUESTED ARTICLE"].nunique()

83

In [10]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

(district_req[district_req["REQUESTED ARTICLE"]=="AHUJA Radios"]["QUANTITY"].to_list()[0])

1

### Public

In [11]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public
...,...,...,...,...,...,...,...,...,...,...,...,...
134,P 135,V.Saranya,349367784825,No,"Melapatty, Madurai",8939219409,Girls Cycle,5200,1,5200,No,Public
135,P 136,Ammu.s,761148632786,No,"Manimangalam, Kancheepuram",8675683781,Renovation Aid,10000,1,10000,No,Public
136,P 137,"Anjali, Thirumalai",261896284949,No,Manimangalam,9043936660,Renovation Aid,10000,1,10000,No,Public
137,P 138,Ananthan,934364951810,Yes,"Vikkiravandi, Villupuram",9159583083,Flood Relief Fund,15000,1,15000,No,Public


In [12]:
public_df["REQUESTED ARTICLE"].nunique()

28

In [13]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [14]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,1
1,Bosch Electrician Kit 13 Re,1
2,Business Aid,20
3,Construction Aid,6
4,Education Aid,8
5,Electronic weighing scale,1
6,Ex gratia for Deceased,1
7,Financial Aid,5
8,Flood Relief Fund,2
9,Gents Cycle,5


### Institution

In [15]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type
0,I 001,Cheyyur Thasildar Office,Desktop Computer,1,45500,45500,"Thasildar Office, Cheyyur, Chengalpattu",0,needs high config PC for e-seva point,Institution
1,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,044 27529096,No,Institution
2,I 003,Govt School - MKM,Steel Cupboard,2,0,0,Marakanam,9444662668,No,Institution
3,I 004,Govt School - KDPM,Office Table 4X2,5,0,0,Kadapakkam,9442826622,No,Institution
4,I 004,Govt School - KDPM,S Type Chair,5,0,0,Kadapakkam,9442826622,No,Institution
5,I 005,Govt.School - AVPR,Office Table 4X2,5,0,0,"Avanipur, Villupuram",8778403593,No,Institution
6,I 006,Govt.School- NMBR,Steel Cupboard,2,0,0,Nolambur,94442375680,No,Institution
7,I 007,Govt.School - CYR,Steel Cupboard,3,0,0,Cheiyur,8339068395,No,Institution
8,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,0,0,Chengalpattu,9865336966,No,Institution


In [16]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [17]:
inst_df["REQUESTED ARTICLE"].nunique()

6

In [18]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,Desktop Computer,1
1,Laptop,81
2,Office Table 4X2,10
3,Rice 1000 Kgs,1
4,S Type Chair,5
5,Steel Cupboard,7


### Others

In [19]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type


In [20]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [21]:
oth_df["REQUESTED ARTICLE"].nunique()

0

In [22]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY


In [23]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,3 Ltr Table Top Wet Grinder,1
1,5 HP Bore well Pump / DOL Starter,2
2,AHUJA Radios,1
3,Agri Battery Sprayer,112
4,Agri Manual Sprayer,33
...,...,...
87,Wet Grinder 3 Ltrs,2
88,Wet Grinder 5 Ltrs,4
89,Wet Grinder Floor Model 2 Ltr,26
90,Wood Saw Machine (P),2


In [24]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,3 Ltr Table Top Wet Grinder,District,1
1,5 HP Bore well Pump / DOL Starter,District,2
2,AHUJA Radios,District,1
3,Agri Battery Sprayer,District,111
4,Agri Battery Sprayer,Public,1
...,...,...,...
112,Wet Grinder 3 Ltrs,Public,1
113,Wet Grinder 5 Ltrs,District,4
114,Wet Grinder Floor Model 2 Ltr,District,26
115,Wood Saw Machine (P),District,2


In [25]:
district_df

,REQUESTED ARTICLE,QUANTITY,Beneficiary Type
0,Agri Battery Sprayer,9,District
1,Canon Printer 6030B (USB),1,District
2,Desktop Computer,1,District
3,Sewing Machine ORD,16,District
4,Agri Power Sprayer (2 STK),2,District
...,...,...,...
383,Ex gratia for Deceased,1,District
384,Medical Aid,1,District
385,Provision materials to Orphanages,1,District
386,Bosch Rotary Hammer GBH 220,1,District


In [26]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13246
Public QTY 140
Inst&Oth QTY 105
total qty - merge 13491
total qty - added 13491


In [27]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Public,Others,Total,Ordered Quantity,Remaining Quantity
0,3 Ltr Table Top Wet Grinder,1,0,0,0,1,0,1
1,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,0,2
2,AHUJA Radios,1,0,0,0,1,0,1
3,Agri Battery Sprayer,111,0,1,0,112,0,110
4,Agri Manual Sprayer,33,0,0,0,33,0,33
...,...,...,...,...,...,...,...,...
87,Wet Grinder 3 Ltrs,1,0,1,0,2,0,2
88,Wet Grinder 5 Ltrs,4,0,0,0,4,0,4
89,Wet Grinder Floor Model 2 Ltr,26,0,0,0,26,0,26
90,Wood Saw Machine (P),2,0,0,0,2,0,2


In [28]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution&oth Total",updated_df["Institution"].sum())


District Total 13246
Public Total 140
Institution&oth Total 105


In [29]:
updated_df["Total"].sum()

np.int64(13491)

In [50]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,NaN,NaN,No,District
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,1200.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
383,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District
384,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District
385,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District
386,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District


In [53]:
district_df["App. No."]=""
cnt = 1
for names in district_df["NAME OF THE DISTRICT"].unique():
    district_df.loc[district_df["NAME OF THE DISTRICT"] == names, "App. No."] = "D " + str(f"{cnt:03}")
    cnt += 1

In [54]:
district_df[["App. No.","NAME OF THE DISTRICT","REQUESTED ARTICLE",""]]

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type,App. No.
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District,D 001
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,NaN,NaN,No,District,D 001
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District,D 001
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,1200.0,No,District,D 001
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District,D 002
...,...,...,...,...,...,...,...,...,...,...,...
383,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District,D 060
384,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District,D 060
385,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District,D 060
386,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District,D 061


In [56]:
public_df = read_file(public_master_id)

public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,P 135,V.Saranya,349367784825,No,"Melapatty, Madurai",8939219409,Girls Cycle,5200,1,5200,No,Public,Article
135,P 136,Ammu.s,761148632786,No,"Manimangalam, Kancheepuram",8675683781,Renovation Aid,10000,1,10000,No,Public,Aid
136,P 137,"Anjali, Thirumalai",261896284949,No,Manimangalam,9043936660,Renovation Aid,10000,1,10000,No,Public,Aid
137,P 138,Ananthan,934364951810,Yes,"Vikkiravandi, Villupuram",9159583083,Flood Relief Fund,15000,1,15000,No,Public,Project


In [35]:
inst_df = read_file(inst_data_id)
inst_df

Download progress: 100%


,App. No.,Institution Name,Article Name,Quantity,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type
0,I 001,Cheyyur Thasildar Office,Desktop Computer,1,45500,45500,"Thasildar Office, Cheyyur, Chengalpattu",0,needs high config PC for e-seva point,Institution
1,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,044 27529096,No,Institution
2,I 003,Govt School - MKM,Steel Cupboard,2,0,0,Marakanam,9444662668,No,Institution
3,I 004,Govt School - KDPM,Office Table 4X2,5,0,0,Kadapakkam,9442826622,No,Institution
4,I 004,Govt School - KDPM,S Type Chair,5,0,0,Kadapakkam,9442826622,No,Institution
5,I 005,Govt.School - AVPR,Office Table 4X2,5,0,0,"Avanipur, Villupuram",8778403593,No,Institution
6,I 006,Govt.School- NMBR,Steel Cupboard,2,0,0,Nolambur,94442375680,No,Institution
7,I 007,Govt.School - CYR,Steel Cupboard,3,0,0,Cheiyur,8339068395,No,Institution
8,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,0,0,Chengalpattu,9865336966,No,Institution
